In [13]:
import pandas as pd
import numpy as np
import pickle

#稀疏矩阵，打分表
import scipy.io as sio
import os

#距离
import scipy.spatial.distance as ssd

In [14]:
#用户和歌曲的索引
users_index = pickle.load(open("users_index.pkl", 'rb'))
songs_index = pickle.load(open("songs_index.pkl", 'rb'))

#训练集上用户的数目和歌曲的数目
num_users = len(users_index)
num_songs = len(songs_index)

#用户听过的歌曲
user_song_dict = pickle.load(open('user_song_dict.pkl', 'rb'))
#歌曲被听过的用户
song_user_dict = pickle.load(open('song_user_dict.pkl', 'rb'))

In [15]:
#使用用户的记录
user_song_interest = sio.mmread("user_song_interest")#.todense()
user_song_interest = user_song_interest.tocsr()

In [16]:
#使用用户的打分
user_song_score = sio.mmread("user_song_score")#.todense()
user_song_score = user_song_score.tocsr()

In [32]:
users_mean = np.zeros(num_users)
for u in range(num_users):  
    n_user_items = 0
    r_acc = 0.0
    if len(user_song_dict[u]) == 0:
        users_mean[u] = 0
        continue
    for i in user_song_dict[u]:  #用户打过分的item
        r_acc += user_song_score[u,i]
        n_user_items += 1
 
    users_mean[u] = r_acc/n_user_items

In [36]:
#计算歌曲之间的相似度
def songs_similarity(song_x, song_y):
    su={}  #有效item（两个用户均有打分的item）的集合
    for user in song_user_dict[song_x]:  #对iid1所有打过分的用户
        if user in song_user_dict[song_y]:  #如果该用户对iid2也打过分
            su[user]=1  #该用户为一个有效user
        
    n=len(su)   #有效item数，有效item为即对uid对Item打过分，uid2也对Item打过分
    if (n==0):  #没有共同打过分的item，相似度设为0？
        similarity=0  
        return similarity  
        
    #iid1的有效打分(减去用户的平均打分)
    s1=np.array([user_song_score[user,song_x]-users_mean[user] for user in su])
        
    #iid2的有效打分(减去用户的平均打分)
    s2=np.array([user_song_score[user,song_y]-users_mean[user] for user in su])  
    
    similarity = 1 - ssd.cosine(s1, s2) 
    if( np.isnan(similarity) ):  #分母为0（s1或s2中所有元素为0）
        similarity = 0.0
    if similarity != 0.0:
        print(similarity)
    return similarity 
songs_similarity(50, 30)

D:\soft\Anaconda\lib\site-packages\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


0.0

In [37]:
#计算所有歌曲之间的相似性
songs_similarity_matrix = np.matrix(np.zeros(shape=(num_songs, num_songs)), float)

for i in range(num_songs):
    songs_similarity_matrix[i,i] = 1.0
    
    #打印进度条
    if(i % 100 == 0):
        print ("i=%d " % (i) )

    for j in range(i+1,num_songs):   #items by user 
        songs_similarity_matrix[j,i] = songs_similarity(i, j)
        songs_similarity_matrix[i,j] = songs_similarity_matrix[j,i]

pickle.dump(songs_similarity_matrix, open("songs_similarity_matrix.pkl", 'wb')) 

i=0 


D:\soft\Anaconda\lib\site-packages\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


i=100 
i=200 
i=300 
i=400 
i=500 
i=600 
i=700 


In [52]:
#对物品进行预测
def Item_CF_pred(uid, iid): 
    sim_accumulate=0.0  
    rat_acc=0.0 
    
    for item_id in user_song_dict[uid]:  #对用户打过分的item
        #计算当前用户打过分item与其他item之间的相似度
        #sim = item_similarity(item_id, iid)
        sim = songs_similarity_matrix[item_id, iid]
        
        #由于相似性可能为负，而用户打过分的item又不多（与iid不相似的item占多数）预测打分为负
        if sim != 0: 
            rat_acc += sim * (user_song_score[uid, item_id])   #用户user对item i的打分
            sim_accumulate += np.abs(sim)                 
        
    if sim_accumulate != 0: #no same user rated,return average rates of the data  
        score = rat_acc/sim_accumulate
    else:   #no items the same user rated,return average rates of the user  
        score = users_mean[uid]
    
    if score <0:
        score = 0.0
    
    return score

In [54]:
#对用户进行推荐
def recommend(user):
    cur_user_id = users_index[user]
    
    #训练集中该用户打过分的item
    cur_user_items = user_song_dict[cur_user_id]

    #该用户对所有item的打分
    user_items_scores = np.zeros(num_songs)

    #预测打分
    for i in range(num_songs):  # all items 
        if i not in cur_user_items: #训练集中没打过分
            user_items_scores[i] = Item_CF_pred(cur_user_id, i)  #预测打分
    
    #推荐
    #Sort the indices of user_item_scores based upon their value，Also maintain the corresponding score
    sort_index = sorted(((e,i) for i,e in enumerate(list(user_items_scores))), reverse=True)
    
    #Create a dataframe from the following
    columns = ['item', 'score']
    df = pd.DataFrame(columns=columns)
         
    #Fill the dataframe with top 20 (n_rec_items) item based recommendations
    #sort_index = sort_index[0:n_rec_items]
    #Fill the dataframe with all items based recommendations
    for i in range(0,len(sort_index)):
        cur_item_index = sort_index[i][1] 
        cur_item = list (songs_index.keys()) [list (songs_index.values()).index (cur_item_index)]
            
        if ~np.isnan(sort_index[i][0]) and cur_item_index not in cur_user_items:
            df.loc[len(df)]=[cur_item, sort_index[i][0]]
    
    return df

In [43]:
#读取测试数据
test_data = pd.read_csv('test_data.csv')
test_data.head()

,Unnamed: 0,user,song,play_count,interest,score
0,28439,f8181f9b3d85fa4ac04c66bc9f84f0ad2a18a777,SOPUELG12A6701D215,4,1,1
1,20964,6bfaf522b9bfb59370d8adfa7f6093579502d775,SOVVENS12A8C144DB5,17,1,1
2,27315,002b63a7e2247de6d62bc62f253474edc7dd044c,SOMCMKG12A8C1347BF,6,1,1
3,12408,fe9a05c03c29da973743a83b80d1660748077432,SOMDVSL12A6D4F7230,8,1,1
4,2749,1820cfffd52cad7b3af398f379524d51579655d2,SOWIMTL12A8C1386DC,1,0,1


In [57]:
#统计总的用户
unique_users_test = test_data['user'].unique()

#为每个用户推荐的item的数目
n_rec_items = 10

#性能评价参数初始化，用户计算Percison和Recall
n_hits = 0
n_total_rec_items = 0
n_test_items = 0

#所有被推荐商品的集合（对不同用户），用于计算覆盖度
all_rec_items = set()

#残差平方和，用与计算RMSE
rss_test = 0.0

#对每个测试用户
for user in unique_users_test:
    #测试集中该用户打过分的电影（用于计算评价指标的真实值）
    if user not in users_index:   #user在训练集中没有出现过，新用户不能用协同过滤
        print(str(user) + ' is a new user.\n')
        continue
   
    user_records_test= test_data[test_data.user == user]
    
    #对每个测试用户，计算该用户对训练集中未出现过的商品的打分，并基于该打分进行推荐（top n_rec_items）
    #返回结果为DataFrame
    rec_items = recommend(user)
    
    for i in range(n_rec_items):
        item = rec_items.iloc[i]['item']
        
        if item in user_records_test['song'].values:
            n_hits += 1
        all_rec_items.add(item)
    
    #推荐的item总数
    n_total_rec_items += n_rec_items
    
    #真实item的总数
    n_test_items += user_records_test.shape[0]

#Precision & Recall
precision = n_hits / (1.0*n_total_rec_items)
recall = n_hits / (1.0*n_test_items)
print('precision:{}, recall:{}'.format(precision, recall))

b1054cf00f4b3d2b7fc1548b19e423a50639c22f is a new user.

bb0d8c40d8932c5960e8e4443aa391adb9dfe9da is a new user.

de27b74444dae039f76e421362c6a914da9f8b41 is a new user.

precision:0.004347826086956522, recall:0.004266097853619517
